In [5]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import re
import stanza
from collections import Counter
stanza.download('en')  

2020-04-30 23:56:01 INFO: Downloading default packages for language: en (English)...
2020-04-30 23:56:02 INFO: File exists: /home/tdimson/stanza_resources/en/default.zip.
2020-04-30 23:56:05 INFO: Finished downloading models and saved to /home/tdimson/stanza_resources.


In [7]:
import itertools
import datasets
import pickle
from transformers import AutoModelWithLMHead, AutoTokenizer

In [8]:

def print_words(words, f):
    for word in words:
        word_str = [word.word]
        if word.pos:
            word_str.append(f"/{word.pos}/")
        if word.topic:
            word_str.append(f"[{word.topic}]")
        print(" ".join(word_str), file=f)
        print(f"\t{word.definition}{' |n| ' if word.example is None else ''}", file=f)
        if word.example:
            print(f"\t\"{word.example}\"{' |e|' if word.from_example_expansion else ''}", file=f)

        print("", file=f)

2020-04-28 23:37:24 WARNING: Can not find mwt: default from official model list. Ignoring it.
2020-04-28 23:37:24 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |
| pos       | ewt     |

2020-04-28 23:37:24 INFO: Use device: gpu
2020-04-28 23:37:24 INFO: Loading: tokenize
2020-04-28 23:37:24 INFO: Loading: pos
2020-04-28 23:37:25 INFO: Done loading processors!


Counter({'VBD': 3, 'NNP': 0, 'VBN': 0, 'IN': 0, 'POS': 0, 'NN': 0, '.': 0, 'PRP': 0, 'CD': 0, 'CC': 0, 'VBZ': 0})


'VBD'

In [9]:
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos')
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens(datasets.SpecialTokens.special_tokens_dict())
blacklist = datasets.Blacklist.load("../data/blacklist.pickle")
model = AutoModelWithLMHead.from_pretrained("../models/inverse_en_dictionary_parsed_lr_00001/checkpoint-200000").to("cuda:1")

2020-04-30 23:56:07 WARNING: Can not find mwt: default from official model list. Ignoring it.
2020-04-30 23:56:07 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |
| pos       | ewt     |

2020-04-30 23:56:07 INFO: Use device: gpu
2020-04-30 23:56:07 INFO: Loading: tokenize
2020-04-30 23:56:09 INFO: Loading: pos
2020-04-30 23:56:10 INFO: Done loading processors!


'<|bod|>the feeling of falling perpetually into the ground.<|bd|>'

In [18]:
defn = "a programmer who can't seem to figure out what is wrong with his code"
prefix = f"{datasets.SpecialTokens.BOS_TOKEN}{defn}{datasets.SpecialTokens.DEFINITION_SEP}"
words, stats = datasets.InverseParsedDictionaryDefinitionDataset.generate_words(
    tokenizer, model,
    prefix=prefix,
    num=100,
    max_iterations=1, 
    blacklist=blacklist, 
    generation_args=dict(
        top_k=500,
        num_return_sequences=50,
        max_length=256,
        do_sample=True,
    ),
)

print_words(words, sys.stdout)

sickle-picker /noun/
	a programmer who can't seem to figure out what is wrong with his code
	"a programmer hit a big white mark by repeatedly destroying source code within minutes of its release"

trivialist /noun/
	a programmer who can't seem to figure out what is wrong with his code
	"his basic theorem proved that the vector space allows two int and two floats to be vectors and is fixed"

hard coded /noun/
	a programmer who can't seem to figure out what is wrong with his code
	"the developer is a known hacker, and may be manually programmed to do the job"

cynogandist /noun/
	a programmer who can't seem to figure out what is wrong with his code
	"the software can't read code that is designed to be gibberish"

nickwaggle /noun/
	a programmer who can't seem to figure out what is wrong with his code
	"the hacker behind the hack is back as anickwagler"

bugcatcher /noun/
	a programmer who can't seem to figure out what is wrong with his code
	"especially bugcatchers don't know how to writ